In [1]:
from keras.models import Sequential
from keras.layers import *
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras


from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
#划分数据集
def generate_classification_train_data():

    data = pd.read_csv('train.csv')
    data = data.iloc[:,2:]
    # 预处理
    data.iloc[:,-1] = data.iloc[:,-1].replace(["Fair", "Good", "Very Good","Premium", "Ideal"],[0,1,2,3,4])
    print(data)

    data['color'] = data['color'].replace(["J","I","H", "G", "F","E", "D"],[1,2,3,4,5,6,7])
    print(data)

    data['clarity'] = data['clarity'].replace(["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"],[1,2,3,4,5,6,7,8])
    x=data.iloc[:,0:-1].values
    y=data.iloc[:,-1].values

    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2,random_state=0)


    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test

#Seque构建方式（推荐）
# class SequeClassifier():
#     def __init__(self, units):
#         self.units = units
#         self.model = None
#
#     #构建神经网络模型：（根据各层输入输出的shape）搭建网络结构、确定损失函数、确定优化器
#     def build_model(self, loss, optimizer, metrics):
#         self.model = Sequential()
#         self.model.add(LSTM(self.units, return_sequences=True))
#         self.model.add(Dropout(0.2))
#         self.model.add(LSTM(self.units))
#         self.model.add(Dropout(0.2))
#         self.model.add(Dense(5, activation='softmax')) #最后一层全连接层。对于N分类问题，最后一层全连接输出个数为N个，这里鸢尾花数据为3分类问题；
#
#         self.model.compile(loss=loss,
#                            optimizer=optimizer,
#                            metrics=metrics)

model = keras.models.Sequential()
model.add(keras.layers.LSTM(units = 128, return_sequences = True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(units=128))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(5, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = 'adam',
              metrics = ['accuracy'])

In [3]:
#1 获取训练数据集，并调整为三维输入格式
x_train, y_train, x_test, y_test = generate_classification_train_data()
print(type(x_test))

      carat  depth  table  price     x     y     z color clarity  label
0      0.71   61.1   54.0   4937  5.78  5.84  3.55     F    VVS1      4
1      0.32   61.3   55.0    707  4.45  4.43  2.72     G     VS1      4
2      1.07   61.8   61.0   5872  6.57  6.54  4.05     D     SI1      3
3      0.41   60.0   56.0    863  4.82  4.88  2.91     F     VS2      4
4      0.53   62.3   57.0   1956  5.23  5.19  3.25     E     VS2      4
...     ...    ...    ...    ...   ...   ...   ...   ...     ...    ...
8626   0.70   60.6   57.0   1919  5.73  5.78  3.49     H     SI1      4
8627   0.40   61.6   57.0   1092  4.73  4.75  2.92     E     VS2      4
8628   1.22   62.3   56.0  10221  6.84  6.81  4.25     G    VVS2      4
8629   0.50   62.3   54.0   1279  5.11  5.13  3.19     H     VS2      4
8630   0.34   62.6   58.0    745  4.43  4.45  2.78     E     VS2      2

[8631 rows x 10 columns]
      carat  depth  table  price     x     y     z  color clarity  label
0      0.71   61.1   54.0   4937  5.7

In [4]:
x_train = x_train[:, :, np.newaxis]
x_test = x_test[:, :, np.newaxis]
x_train


array([[[ 1.  ],
        [62.3 ],
        [59.  ],
        ...,
        [ 3.91],
        [ 7.  ],
        [ 3.  ]],

       [[ 0.41],
        [61.3 ],
        [55.  ],
        ...,
        [ 2.93],
        [ 6.  ],
        [ 7.  ]],

       [[ 1.73],
        [65.9 ],
        [58.  ],
        ...,
        [ 4.93],
        [ 3.  ],
        [ 2.  ]],

       ...,

       [[ 1.02],
        [63.6 ],
        [58.  ],
        ...,
        [ 4.05],
        [ 4.  ],
        [ 3.  ]],

       [[ 0.32],
        [59.9 ],
        [55.  ],
        ...,
        [ 2.68],
        [ 6.  ],
        [ 5.  ]],

       [[ 1.03],
        [61.7 ],
        [56.  ],
        ...,
        [ 4.  ],
        [ 4.  ],
        [ 7.  ]]])

In [5]:
#2 构建神经网络模型：（根据各层输入输出的shape）搭建网络结构、确定损失函数、确定优化器
units = 128 #lstm细胞个数
loss = "sparse_categorical_crossentropy"  #损失函数类型
optimizer = "adam"  #优化器类型
metrics = ['accuracy']  #评估方法类型
# sclstm = SequeClassifier(units)
# sclstm.build_model(loss, optimizer, metrics)

callbacks = [

    #大家可以继续调小min_delta，默认监控的指标是验证集的monitor='val_loss'
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)
]

#3 训练模型
epochs = 100
batch_size = 32
history = model.fit(x_train, y_train, epochs=epochs,
                 batch_size=batch_size,
                 callbacks = callbacks)

#4 模型评估
# score = sclstm.model.evaluate(x_test, y_test, batch_size=32)
# print("model score:", score)

# 模型应用：预测
#proba_prediction = sclstm.model.predict(x_test)

#5 模型持久化，把模型保存在本地
dirs = "model"
if not os.path.exists(dirs):
    os.makedirs(dirs)
print("正在保存模型......")

model.save(dirs+"/classifier_model.h5")
print("模型已保存.save path-->dirs%s"%"/classifier_model.h5")

Epoch 1/100
216/216 [==============================] - 6s 12ms/step - loss: 1.3858 - accuracy: 0.4002
Epoch 2/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3632 - accuracy: 0.4019
Epoch 3/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3546 - accuracy: 0.4070
Epoch 4/100
216/216 [==============================] - 3s 12ms/step - loss: 1.3528 - accuracy: 0.4112
Epoch 5/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3502 - accuracy: 0.4118
Epoch 6/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3486 - accuracy: 0.4067
Epoch 7/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3481 - accuracy: 0.4102
Epoch 8/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3433 - accuracy: 0.4153
Epoch 9/100
216/216 [==============================] - 2s 11ms/step - loss: 1.3331 - accuracy: 0.4280
Epoch 10/100
216/216 [==============================] - 3s 13ms/step - loss: 1.335

In [6]:
history.history

{'loss': [1.3857722282409668,
  1.3632385730743408,
  1.3545681238174438,
  1.3527588844299316,
  1.350156307220459,
  1.3485544919967651,
  1.3480788469314575,
  1.3432565927505493,
  1.3331409692764282,
  1.3349641561508179,
  1.2306009531021118,
  1.196376919746399,
  1.1910902261734009,
  1.1459681987762451,
  1.1439919471740723,
  1.1178784370422363,
  1.0996636152267456,
  1.0972707271575928,
  1.1011899709701538,
  1.0905498266220093,
  1.1091009378433228,
  1.0935025215148926,
  1.0828124284744263,
  1.084281325340271,
  1.0782439708709717,
  1.0724427700042725,
  1.0976516008377075,
  1.062820553779602,
  1.0732700824737549,
  1.0828810930252075,
  1.0598344802856445,
  1.0738452672958374,
  1.0603822469711304,
  1.0662232637405396,
  1.056316614151001,
  1.0682272911071777,
  1.044029951095581,
  1.0486830472946167,
  1.0501350164413452,
  1.0397919416427612,
  1.0558674335479736,
  1.03879714012146,
  1.0424792766571045,
  1.0169602632522583,
  1.0379489660263062,
  1.006108

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 9, 128)            66560     
                                                                 
 dropout (Dropout)           (None, 9, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 202,437
Trainable params: 202,437
Non-trai

In [7]:
#6 从指定模型保存的位置读取模型，做预测
from keras.models import load_model
read_model = load_model(dirs+"/classifier_model.h5")
out = read_model.predict(x_test)
print("out:%s"%out)


out = tf.nn.softmax(out)
out = np.array(out)

pre_test = np.argmax(out,axis=1)
pre_test

out:[[1.7493175e-04 4.9531285e-02 2.5250018e-01 4.8212650e-01 2.1566711e-01]
 [3.3775042e-03 2.3632723e-01 6.9842303e-01 5.6251142e-02 5.6211082e-03]
 [3.6270421e-06 6.1643552e-03 1.6697787e-01 5.3884920e-02 7.7296919e-01]
 ...
 [3.6824646e-04 2.8949821e-01 4.1128907e-01 6.7367278e-02 2.3147716e-01]
 [1.1973706e-02 4.3565997e-01 5.0591010e-01 1.0185913e-02 3.6270302e-02]
 [7.8017973e-07 1.2498940e-03 9.5556930e-02 3.3870790e-02 8.6932158e-01]]


array([3, 2, 4, ..., 2, 2, 4], dtype=int64)

In [8]:
# 测单个
color = 'H'
clarity = 'VVS1'
color_dict = {"J":1, "I":2, "H":3, "G":4, "F":5, "E":6, "D":7}
color_num = color_dict[color]

clarity_dict = {"I1":1, "SI2":2, "SI1":3, "VS2":4, "VS1":5, "VVS2":6, "VVS1":7, "IF":8}
clarity_num = clarity_dict[clarity]

test = np.array([[0.32,62.5,58,730,4.33,4.37,2.72,color_num, clarity_num]])
print('test',test)
# test = np.array([[0.32,62.5,58,730,4.33,4.37,2.72,3,7]])
test = test[:, :, np.newaxis]
out = read_model.predict(test)
print("out:%s"%out)


out = tf.nn.softmax(out)
out = np.array(out)

pre_test = np.argmax(out,axis=1)
print(pre_test[0])
print(type(int(pre_test[0])))

all_label_list = ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']
final = int(pre_test[0])
finalresult = str(all_label_list[4-final])
print(finalresult)

test [[3.20e-01 6.25e+01 5.80e+01 7.30e+02 4.33e+00 4.37e+00 2.72e+00 3.00e+00
  7.00e+00]]
out:[[2.0022268e-05 1.5176520e-02 3.5305688e-01 4.5503917e-01 1.7670739e-01]]
3
<class 'int'>
Premium
